# Initial AI Agent PPO w/o


In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 900
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 14,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "simplex"
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-15 12:21:15,588	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=600,
        lr=0,
        )
)

In [6]:
# import json

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         try:
#             json.dumps(obj)
#         except TypeError:
#             return "Not serializable object!"

#         return obj

# config_dict = config.to_dict()

# print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [7]:
#algo = config.build()

In [8]:
# from ray.rllib.policy.policy import Policy

# # Use the `from_checkpoint` utility of the Policy class:
# my_restored_policy = Policy.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-14_22-11-44ayp9l_kd/checkpoint_000881/policies/default_policy")


In [9]:
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-14_22-11-44ayp9l_kd/checkpoint_000881")

2023-10-15 12:21:21,412	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-14_22-11-44ayp9l_kd/checkpoint_000881! Trying to extract checkpoint info from other files found in that dir.
2023-10-15 12:21:21,460	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=4692) GT Real Time instantiated
(RolloutWorker pid=4692) GT AI Server instantiated for rtgym
(RolloutWorker pid=4692) starting up on localhost port 9999
(RolloutWorker pid=4692) Waiting for a connection
(RolloutWorker pid=4692) Connection from ('127.0.0.1', 59559)
(RolloutWorker pid=4692) J:\git\TensorFlowPSX\Py\dragSpaced.csv
(RolloutWorker pid=4692) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2


2023-10-15 12:21:36,698	INFO trainable.py:172 -- Trainable.setup took 15.239 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [11]:
for i in range(5):
    result = algo.train() #single try with lr=0
#algo.evaluate()

ValueError: Cannot evaluate w/o an evaluation worker set in the Trainer or w/o an env on the local worker!
Try one of the following:
1) Set `evaluation_interval` >= 0 to force creating a separate evaluation worker set.
2) Set `create_env_on_driver=True` to force the local (non-eval) worker to have an environment to evaluate on.